# Extração da característica de relação sinal-ruído de dados de EEG

A ideia é utilizar dados fictícios de ruído e sinal "bom"., para criarmos a relação dos dois sinais e obter como resultado um sinal de interesse "limpo".

A partir deste sinal, podemos no contexto de caracterização de foco, ainda extrair os rítmos cerebrais ou então classificar sinais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta atividade pode ser realizada em conjunto com um classificador comumente utilizado, como é o caso do SVM. Neste caso, uma porcetagem das amostras são utilizadas para treino e o restante para teste (p.e. 30 e 70% respectivamente).

In [3]:
# Criando dados fictícios
# pensando em um vídeo de 10 minutos, com uma taxa de amostragem de 250 Hz
#   teremos 10 * 60 * 250 = 150.000 amostras (sinal com foco)
#   teremos 1 * 60 * 250 = 15.000 amostras (basal)

import numpy as np


shape = (150000, 8)
data_focus = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

shape = (15000, 8)
data_base = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

In [6]:
# Estimando o ruído de fundo (utilizando o sinal basal)

# armazena uma lista com as médias de potência para cada canal
noise_power = []
for channel_data in data_base.T:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(psd)
    noise_power.append(base_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

1507415.2


Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [9]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse

# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = data_focus - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)


[[        nan  9.401269           nan ...         nan  2.9831462
          nan]
 [        nan         nan         nan ...  9.281207           nan
          nan]
 [        nan  8.027126   10.173944   ...  9.4063635          nan
   9.69626   ]
 ...
 [ 1.614929   -0.05026207  5.9073105  ...         nan  9.621689
   6.9831777 ]
 [13.913423    3.1487334          nan ...         nan 12.691887
  12.565821  ]
 [        nan         nan         nan ...  1.2228951  -2.109276
   0.10015392]]
(150000, 8)
[[-52.173462        nan -55.80818  ... -47.119846        nan -51.672825]
 [-53.213123 -54.71786  -58.384212 ...        nan -52.377968 -51.459293]
 [-54.411148        nan        nan ...        nan -48.132893        nan]
 ...
 [       nan        nan        nan ... -47.17713         nan        nan]
 [       nan        nan -50.74326  ... -52.89728         nan        nan]
 [-55.696777 -50.52421  -60.08892  ...        nan        nan        nan]]
(150000, 8)


/tmp/ipykernel_6508/1008779610.py:9: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
/tmp/ipykernel_6508/1008779610.py:14: RuntimeWarning: invalid value encountered in log10
  wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)


### Tarefa para aplicação das características SNR:

Agora que temos os dois vetores de características SNR, podemos utilizar buffers com e sem a evocação dos rítmos que caracterizam o foco.

Fiquem livres para testar essas característica utilizando o formato empírico visto até o momento ou criando dados de treinamento e teste para serem submetidos a um classificador.